In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['yelp-dataset', 'balanced-reviews']


In [2]:
reviews_final = pd.read_csv("../input/balanced-reviews/balanced_reviews.csv", index_col=0)

In [3]:
df_reviews = reviews_final[['funny','text', 'fun_bin']]

In [4]:
pd.set_option('display.max_colwidth', -1)
df_reviews.head()

,funny,text,fun_bin
17,7,"I love chinese food and I love mexican food. What can go wrong? A couple of things. First things first, this place is more of a ""rice bowl"" kind of place. I thought it was going to be more diverse as far as the menu goes, but its mainly rice bowls you get with different kinds of meats. The ordering was a little confusing at first, but one of the employees helped us out and I got the 2-item bowl and got the jade chicken and hengrenade chicken with all rice(jerk). I also ordered a jade chicken quesadilla on the side.\n\nI'm gonna admit, this place looks kinda dirty. I don't think Arizona uses those health department letter grade system like California does, but if I were to just judge by how it looked inside, i'd give it a ""C"" grade lol. We waited for about 15 minutes or so and finally got our food. We took it to go and ate at our hotel room. \n\nMmmm... the food was just alright. The jade chicken was nothing special. It tasted like any generic chinese fast food orange chicken/sesame chicken variant. The hengrenade chicken, although was the less spicier version of the jerk chicken, was still pretty spicy for me. Just be warned the jerk chicken is super spicy. If you aren't sure, ask for a sample at the restaurant before ordering, but it was way too spicy for me. \n\nThe jade chicken quesadilla was decent, but nothing special. Just imagine orange chicken in between a tortilla and cheese. A friend of mine ordered a jade chicken burrito and we were confused when we pulled it out of the bag because it was literally the size of Mcdonald's apple pie. If you order the burrito, be warned that it's a burrito for gnomes and smurfs, but he said it was tasty. \n\nThey provide a snicker doodle sugar cookie for each meal and it was decent, again nothing special. \n\nNot gonna lie, the next day my stomach felt like a little mexican dude and chinese dude were wrestling and throwing molotov cocktails inside. I used the bathroom like 5 times. I don't recommend eating this place if you have a lot to do the next day.",1
21,6,"If you are looking for the best pierogies in Pittsburgh, this is your place. There are a few small tables outside but most of the business is carry out. Pierogies Plus wins Best Pierogies every year. Why? Because the owner is from Poland and she is making the real deal pierogies. The best part is that they are hand pinched by a group of older Polish and Hungarian women. \nThe biggest seller is potato and cheese but they sell many flavors. They are like plump pillows of softness. You can buy them buy the dozen. You can get them cold to take home and freeze or warm and ready to eat. The warm ones are served with butter and onions. It's definitely a comfort food. The best part is that they ship internationally. Yes, they are that good.",1
62,5,"So good! They didn't make it to 5 stars due to the prices are a bit high for the amount of food and the location is a bit unsavory. \nThe decor and atmosphere was surprisingly nice, from the outside I expected to be more run down inside. The staff was very nice. We were surprised how empty the dining room was for a Friday evening.\nWe got Vegetable Samosas to start then ordered Chicken Tikka Masala, Lamb Rogan Josh, rice and plain Naan. Our only complaint was the lamb could've been more tender but everything was flavorful and delicious. \nI would definitely go again if given the chance.",1
126,5,"While the prices are a bit high for a make-your-own pizza, the taste makes up for it. I love going to Seventh Street market, sitting Not Just Coffee and having a drink while waiting for delicious fresh made pizza from Pure. I've taken this to go as well as eaten inside the market, and I can say that the pizza doesn't do well reheated. So try to eat it fresh while there if possible.\n\nIf one of their specialty pizzas sounds good to you, go for it, as those are definitely a better deal for the amount of toppings you get for the money. I wanted what I wanted, though, so

In [5]:
import pandas as pd
import numpy as np
#import nltk
#from nltk.corpus import stopwords
#from nltk.stem import SnowballStemmer

from string import punctuation


import numpy as np 
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from spacy.symbols import ORTH
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence 
#from spacymoji import Emoji

In [6]:

re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
def sub_br(x): return re_br.sub("\n", x)

#nlp = spacy.load("en")
spacy.load('en')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS



def clean_text(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    #text = text.lower()

    # Clean the text
   # text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=\(\)]", " ", text) # keep punctuatuin, numnbers and letters
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " . ", text) #Add space to the dot
    text = re.sub(r"!", " ! ", text) #Add space to the exclamation sign
    text = re.sub(r":", " :", text) #Add space before : sign
    text = re.sub(r";", " ;", text) #Add space before ; sign
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    #text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    #text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    # find emojis
    emoji_list = []
    '''
    for word in text.split():
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)
    emoji_list'''
    #text = text.split()

    return text

my_tok = spacy.load('en_core_web_sm')
#emoji = Emoji(my_tok)
#my_tok.add_pipe(emoji, first=True)
def spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(clean_text(x))]

def remove_stop_words(tokens): return [tok for tok in tokens if tok not in spacy_stopwords]


### Building a vocabulary

In [7]:
from collections import Counter
counts = Counter()
for sent in df_reviews['text']:
    try:
        counts.update(remove_stop_words(spacy_tok(sent)))
    except:
        pass

In [8]:
#counts

### Vocabulary

In [9]:
# Vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [10]:
# WHat is the 99% quantile of  length of the sentence?
df_reviews['len_text'] = df_reviews['text'].apply(lambda x: len(x.split()))


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
df_reviews['len_text'].quantile(0.99)

769.0

In [12]:
df_reviews['len_text'].quantile(0.95)

496.0

In [13]:
text = "I'm soooo excited!!!!!This is 10000% the best place on earth:))))) 😃..."
text

"I'm soooo excited!!!!!This is 10000% the best place on earth:))))) 😃..."

In [14]:
clean_text(text)

'I m soooo excited ! ! ! ! ! This is 10000% the best place on earth :))))) 😃 . . . '

In [15]:
#spacy_tok(text)

['I',
 'm',
 'soooo',
 'excited',
 '!',
 '!',
 '!',
 '!',
 '!',
 'This',
 'is',
 '10000',
 '%',
 'the',
 'best',
 'place',
 'on',
 'earth',
 ':',
 ')',
 ')',
 ')',
 ')',
 ')',
 '😃',
 '.',
 '.',
 '.']

In [16]:
#remove_stop_words(spacy_tok(text))

['I',
 'm',
 'soooo',
 'excited',
 '!',
 '!',
 '!',
 '!',
 '!',
 'This',
 '10000',
 '%',
 'best',
 'place',
 'earth',
 ':',
 ')',
 ')',
 ')',
 ')',
 ')',
 '😃',
 '.',
 '.',
 '.']

In [17]:
# note that spacy_tok takes a while run it just once
def encode_sentence(sent, vocab2index, N=500, padding_start=True):
    "Encoding a sentence adding padding"
    x = remove_stop_words(spacy_tok(sent))
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in x])
    l = min(N, len(enc1))
    if padding_start:
        enc[:l] = enc1[:l]
    else:
        enc[N-l:] = enc1[:l]
    return enc, l

### Splitting into train and validation sets

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df_reviews['text'], df_reviews['fun_bin'], test_size=0.2, random_state=42)

In [19]:
X_train.reset_index(inplace=True, drop=True)
X_valid.reset_index(inplace=True, drop=True)

In [20]:
y_train.reset_index(inplace=True, drop=True)
y_valid.reset_index(inplace=True, drop=True)

In [21]:
y_train.shape

(120401,)

### Writing a dataset

In [22]:
class YelpDataset(Dataset):
    def __init__(self, df, y, N=400, padding_start=True):
        self.df = df
        self.X = [encode_sentence(sent, vocab2index, N, padding_start) for sent in self.df]
        self.y = y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x, s = self.X[idx]
        return x, s, self.y[idx]


In [23]:
train_ds =  YelpDataset(X_train, y_train, padding_start=False)
valid_ds =  YelpDataset(X_valid, y_valid, padding_start=False)


In [24]:
neg=[]
i=0
for x,s,y in train_ds:
    if s <=0:
        neg.append(i)
    i+=1

neg

[]

In [25]:
X_train.loc[118582,]

'Neither'

In [26]:
X_train.drop(index = 118582, inplace=True)
y_train.drop(index = 118582, inplace=True)
X_train.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)

In [27]:
batch_size = 1000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

### LSTM-CNN

In [28]:
train_ds =  YelpDataset(X_train, y_train, padding_start=False)
valid_ds =  YelpDataset(X_valid, y_valid, padding_start=False)

In [29]:
batch_size = 50
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [30]:
V = vocab_size = len(words)
print(V)

177681


In [31]:
x,s,y = next(iter(train_dl))

In [32]:
x.shape

torch.Size([50, 400])

In [33]:
y.shape

torch.Size([50])

In [34]:
# LSTM
embedding_dim = 300
hidden_dim = 100

embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
lstm = nn.LSTM(300, hidden_dim, batch_first=True)
linear = nn.Linear(hidden_dim, 1)
dropout = nn.Dropout(0.5)
        

In [35]:
x1 = embeddings(x.long())

In [36]:
x1.shape

torch.Size([50, 400, 300])

In [37]:
x2 = dropout(x1)
x2.size()

torch.Size([50, 400, 300])

In [38]:
conv_3 = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=3)
x3 = conv_3(x2.transpose(1,2))
x3.size()

torch.Size([50, 100, 398])

In [39]:
conv_4 = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=4)
conv_5 = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=5)

In [40]:
x4 = conv_4(x2.transpose(1,2))
x5 = conv_5(x2.transpose(1,2))
print(x4.size(), x5.size())

torch.Size([50, 100, 397]) torch.Size([50, 100, 396])


In [41]:
# Combine
# 100 3-gram detectors
x3 = nn.ReLU()(x3)
x3 = nn.MaxPool1d(kernel_size = 398)(x3)
x3.size()

torch.Size([50, 100, 1])

In [42]:
x4 = nn.ReLU()(x4)
x4 = nn.MaxPool1d(kernel_size = 397)(x4)
x4.size()

torch.Size([50, 100, 1])

In [43]:
x5 = nn.ReLU()(x5)
x5 = nn.MaxPool1d(kernel_size = 396)(x5)
x5.size()

torch.Size([50, 100, 1])

In [44]:
# concatenate x3, x4, x5
out = torch.cat([x3, x4, x5], 1)
out.size()

torch.Size([50, 100, 3])

In [45]:
out = out.view(out.size(0), -1)
out.size()

torch.Size([50, 300])

In [46]:
out_pack, (ht, ct) = lstm(out.unsqueeze(1))

In [47]:
out_pack.shape

torch.Size([50, 1, 100])

In [48]:
ht.shape

torch.Size([1, 50, 100])

In [49]:
ht[-1].shape

torch.Size([50, 100])

In [ ]:
# needs to convert x to (batch, hidden_dim, sentence_len)

In [50]:
linear(ht[-1]).size()

torch.Size([50, 1])

In [262]:
class CNN_LSTMModel(nn.Module):
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(CNN_LSTMModel, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        #LSTM layer
        self.lstm = nn.LSTM(300, hidden_dim, batch_first=True)
        #CNN layers
        self.conv_3 = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=3)
        self.conv_4 = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=4)
        self.conv_5 = nn.Conv1d(in_channels=embedding_dim, out_channels=100, kernel_size=5)
        self.dropout = nn.Dropout(p=0.1)
        self.linear = nn.Linear(hidden_dim, 1)
        
    def forward(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        # CNN part
        x = x.transpose(1,2)
        x3 = F.relu(self.conv_3(x))
        x4 = F.relu(self.conv_4(x))
        x5 = F.relu(self.conv_5(x))
        x3 = nn.MaxPool1d(kernel_size = 398)(x3)
        x4 = nn.MaxPool1d(kernel_size = 397)(x4)
        x5 = nn.MaxPool1d(kernel_size = 396)(x5)
        out = torch.cat([x3, x4, x5], 1)
        out = out.view(out.size(0), -1)
         # LSTM part
        out_pack, (ht, ct) = self.lstm(out.unsqueeze(1))
        return self.linear(ht[-1])

In [263]:
def val_metrics(m, val_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, s, y in val_dl:
        x = x.long().cuda()
        y = y.float().unsqueeze(1).cuda()
        y_hat = model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        #print(y_pred)
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

In [264]:
def train_epocs(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.long().cuda()
            y = y.float().unsqueeze(1).cuda()
            y_pred = model(x)
            #print(y.size())
            #print(y_pred.size())
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, val_dl)
        print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

In [265]:
def save_model(m, p): torch.save(m.state_dict(), p)
    
def load_model(m, p): m.load_state_dict(torch.load(p))

In [266]:
batch_size = 500
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [267]:
embedding_dim = 300
hidden_dim = 100

In [268]:
vocab_size = len(words)
print(vocab_size)

model = CNN_LSTMModel(vocab_size, embedding_dim, hidden_dim).cuda()

177681


In [269]:
train_epocs(model, epochs=5, lr=0.1 )

train loss 0.704 val loss 0.693 and val accuracy 0.503
train loss 0.698 val loss 0.699 and val accuracy 0.497
train loss 0.700 val loss 0.709 and val accuracy 0.497
train loss 0.700 val loss 0.701 and val accuracy 0.497
train loss 0.699 val loss 0.693 and val accuracy 0.503


In [270]:
save_model(model, '../model_300_100_5_0.1')

In [271]:
train_epocs(model, epochs=5, lr=0.5)

train loss 0.864 val loss 0.694 and val accuracy 0.503
train loss 0.729 val loss 0.795 and val accuracy 0.503
train loss 0.713 val loss 0.704 and val accuracy 0.503
train loss 0.736 val loss 1.310 and val accuracy 0.497
train loss 0.737 val loss 0.694 and val accuracy 0.497


In [256]:
model1 = CNN_LSTMModel(vocab_size, embedding_dim, hidden_dim).cuda()

In [258]:
train_epocs(model1, epochs=5, lr=0.01 )

train loss 0.695 val loss 0.696 and val accuracy 0.497
train loss 0.694 val loss 0.696 and val accuracy 0.497
train loss 0.694 val loss 0.696 and val accuracy 0.497
train loss 0.694 val loss 0.696 and val accuracy 0.497
train loss 0.694 val loss 0.696 and val accuracy 0.497


In [259]:
save_model(model1, '../model_300_100_5_0.01')

In [272]:
model = CNN_LSTMModel(vocab_size, embedding_dim, hidden_dim).cuda()
train_epocs(model, epochs=5, lr=0.01 )

train loss 0.696 val loss 0.693 and val accuracy 0.503
train loss 0.694 val loss 0.694 and val accuracy 0.497
train loss 0.693 val loss 0.693 and val accuracy 0.503
train loss 0.693 val loss 0.694 and val accuracy 0.503
train loss 0.694 val loss 0.693 and val accuracy 0.503


In [273]:
model = CNN_LSTMModel(vocab_size, embedding_dim, hidden_dim).cuda()

In [274]:
train_epocs(model, epochs=5, lr=0.005)

train loss 0.505 val loss 0.469 and val accuracy 0.784
train loss 0.428 val loss 0.468 and val accuracy 0.765
train loss 0.391 val loss 0.471 and val accuracy 0.784
train loss 0.376 val loss 0.478 and val accuracy 0.769
train loss 0.361 val loss 0.501 and val accuracy 0.779


In [275]:
train_epocs(model, epochs=3, lr=0.0005)

train loss 0.327 val loss 0.512 and val accuracy 0.778
train loss 0.317 val loss 0.516 and val accuracy 0.774
train loss 0.307 val loss 0.514 and val accuracy 0.776


In [276]:
train_epocs(model, epochs=3, lr=0.0001)

train loss 0.298 val loss 0.520 and val accuracy 0.778
train loss 0.296 val loss 0.524 and val accuracy 0.775
train loss 0.294 val loss 0.524 and val accuracy 0.777


In [278]:
model = CNN_LSTMModel(vocab_size, embedding_dim, hidden_dim).cuda()

In [279]:
train_epocs(model, epochs=1, lr=0.005)

train loss 0.503 val loss 0.464 and val accuracy 0.785


In [280]:
train_epocs(model, epochs=1, lr=0.0005)

train loss 0.415 val loss 0.459 and val accuracy 0.789


In [281]:
train_epocs(model, epochs=1, lr=0.0001)

train loss 0.398 val loss 0.458 and val accuracy 0.789


In [282]:
train_epocs(model, epochs=1, lr=0.00005)

train loss 0.393 val loss 0.460 and val accuracy 0.790


In [283]:
train_epocs(model, epochs=1, lr=0.00001)

train loss 0.391 val loss 0.459 and val accuracy 0.789


In [284]:
train_epocs(model, epochs=1, lr=0.00005)

train loss 0.391 val loss 0.459 and val accuracy 0.789


In [285]:
train_epocs(model, epochs=2, lr=0.00005)

train loss 0.389 val loss 0.460 and val accuracy 0.789
train loss 0.386 val loss 0.459 and val accuracy 0.789


In [289]:
train_epocs(model, epochs=1, lr=0.00001)

train loss 0.384 val loss 0.460 and val accuracy 0.789


In [290]:
train_epocs(model, epochs=1, lr=0.00003)

train loss 0.383 val loss 0.460 and val accuracy 0.790


In [291]:
save_model(model, '../model_300_100_1_0.00003')

In [292]:
train_epocs(model, epochs=1, lr=0.00003)

train loss 0.382 val loss 0.460 and val accuracy 0.790


In [293]:
train_epocs(model, epochs=1, lr=0.00003)

train loss 0.381 val loss 0.460 and val accuracy 0.790


In [296]:
save_model(model, '../model_300_100_1_0.00003')